# ETL Pipeline raw-to-silver - Uber Dataset

Este bloco importa todas as bibliotecas Python necessárias para o pipeline, como pandas para manipulação de dados, SQLAlchemy para interação com o banco de dados e logging para registrar o progresso.

In [15]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from sqlalchemy.types import TIMESTAMP, CHAR, Enum, FLOAT, INTEGER, VARCHAR
import logging
import time
import os

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Etapa 0 - Criando Função de Conexão com o Banco

Define a função `connect_to_postgres` que estabelece a conexão com o banco de dados PostgreSQL. Ela usa variáveis de ambiente para as credenciais e se conecta ao `db_host` (definido como 'localhost' neste script).

In [16]:
def connect_to_postgres():
    db_user = os.getenv('POSTGRES_USER', 'admin')
    db_password = os.getenv('POSTGRES_PASSWORD', 'admin')
    db_name = os.getenv('POSTGRES_DB', 'postgres')
    db_host = 'localhost' 
    
    conn_string = f"postgresql://{db_user}:{db_password}@{db_host}/{db_name}"
    
    try:
        logging.info("Tentando conectar ao Postgres...")
        engine = create_engine(conn_string)
        connection = engine.connect()
        logging.info("Conexão com o Postgres estabelecida com sucesso!")
        return engine, connection
    except Exception as e:
        logging.error(f"Falha ao conectar: {e}")
    
    logging.critical("Não foi possível conectar ao banco de dados.")
    return None, None

# Etapa 1 - Extract

O script primeiro tenta estabelecer a conexão com o banco de dados chamando `connect_to_postgres`. Se bem-sucedido, ele lê o arquivo CSV ('uber-dataset.csv') do caminho especificado para um DataFrame pandas e registra o número de linhas lidas.

In [17]:
engine, connection = connect_to_postgres()

if not engine:
    raise RuntimeError("Não foi possível conectar ao banco de dados. Abortando o ETL.")

logging.info("Iniciando Etapa 1: Extract")
raw_path = '../data_layer/raw/uber-dataset.csv'
df_raw = pd.read_csv(raw_path)
logging.info(f"{len(df_raw)} linhas lidas do arquivo CSV.")

2025-11-23 15:04:12,012 - INFO - Tentando conectar ao Postgres...
2025-11-23 15:04:12,033 - INFO - Conexão com o Postgres estabelecida com sucesso!
2025-11-23 15:04:12,035 - INFO - Iniciando Etapa 1: Extract
2025-11-23 15:04:12,623 - INFO - 150000 linhas lidas do arquivo CSV.


# Etapa 2 - Transform

Inicia a Etapa 2 (Transform). Este bloco executa as seguintes transformações específicas nos dados brutos:
* Os nomes das colunas são padronizados para minúsculas e com underscores (ex: "Booking ID" -> "booking_id").
* As colunas `date` e `time` são combinadas em uma única coluna `date_time` do tipo datetime.
* Caracteres de aspas (`"`) são removidos das colunas `booking_id` e `customer_id`.
* Colunas de métricas (`avg_vtat`, `avg_ctat`, `booking_value`, `ride_distance`, `driver_ratings`, `customer_rating`) são convertidas para tipo numérico.
* As colunas `cancelled_rides_by_customer` e `cancelled_rides_by_driver` são consolidadas na nova coluna `cancelled_by`.
* As colunas `reason_for_cancelling_by_customer` e `driver_cancellation_reason` são consolidadas na nova coluna `reason_for_cancelling`.
* O DataFrame final (`df_final`) é criado selecionando apenas as colunas necessárias e renomeando `driver_ratings` para `driver_rating`.

**LÓGICA DE DUPLICATAS (CSV)**: Como último passo desta etapa, o script executa `df_final.drop_duplicates(subset=['booking_id'], keep='first')`. Isso remove quaisquer linhas *do próprio arquivo CSV* que tenham o mesmo `booking_id`, garantindo que apenas registros únicos avancem para a Etapa 3.

In [18]:
logging.info("Iniciando Etapa 2: Transform")
df = df_raw.copy()
df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

df['date_time'] = pd.to_datetime(df['date'] + ' ' + df['time'], errors='coerce')

logging.info("Limpando aspas dos IDs...")
df['booking_id'] = df['booking_id'].str.strip('"')
df['customer_id'] = df['customer_id'].str.strip('"')

numeric_cols = ['avg_vtat', 'avg_ctat', 'booking_value', 'ride_distance', 'driver_ratings', 'customer_rating']

for col in numeric_cols:
    df[col] = (
        pd.to_numeric(df[col].replace("null", None), errors="coerce")
        .fillna(0)
    )

conditions = [
    df['cancelled_rides_by_customer'].notna(),
    df['cancelled_rides_by_driver'].notna()
]
choices = ['customer', 'driver']
df['cancelled_by'] = np.select(conditions, choices, default=None)

df['reason_for_cancelling'] = df['reason_for_cancelling_by_customer'].fillna(df['driver_cancellation_reason'])

final_columns = {
    'date_time': 'dtt',
    'booking_id': 'bkg_id',
    'booking_status': 'bkg_stt',
    'customer_id': 'cus_id',
    'vehicle_type': 'veh_typ',
    'pickup_location': 'pic_loc',
    'drop_location': 'drp_loc',
    'avg_vtat': 'avg_vtt',
    'avg_ctat': 'avg_ctt',
    'cancelled_by': 'ccd_by',
    'reason_for_cancelling': 'rfc',
    'incomplete_rides_reason': 'irr',
    'booking_value': 'bkg_vle',
    'ride_distance': 'rid_dis',
    'driver_ratings': 'drv_rtg', 
    'customer_rating': 'cus_rtg',
    'payment_method': 'pay_mtd'
}

# Seleciona apenas as colunas que vamos usar
df_final = df[final_columns.keys()]

# Renomeia as colunas para o padrão final
df_final = df_final.rename(columns=final_columns)


df_final['bkg_vle'] = df_final['bkg_vle'].astype('Int64')


total_before = len(df_final)
df_final = df_final.drop_duplicates(subset=['bkg_id'], keep='first')
total_after = len(df_final)
if total_before > total_after:
    logging.info(f"Removidas {total_before - total_after} duplicatas encontradas no arquivo CSV (baseado no 'booking_id').")
else:
    logging.info("Nenhuma duplicata encontrada no arquivo CSV.")


logging.info("Transformação concluída. Schema final do DataFrame:")
print(df_final.info())
print(df_final.head())


2025-11-23 15:04:19,403 - INFO - Iniciando Etapa 2: Transform
2025-11-23 15:04:19,524 - INFO - Limpando aspas dos IDs...
2025-11-23 15:04:19,797 - INFO - Removidas 1233 duplicatas encontradas no arquivo CSV (baseado no 'booking_id').
2025-11-23 15:04:19,799 - INFO - Transformação concluída. Schema final do DataFrame:


<class 'pandas.core.frame.DataFrame'>
Index: 148767 entries, 0 to 149999
Data columns (total 17 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   dtt      148767 non-null  datetime64[ns]
 1   bkg_id   148767 non-null  object        
 2   bkg_stt  148767 non-null  object        
 3   cus_id   148767 non-null  object        
 4   veh_typ  148767 non-null  object        
 5   pic_loc  148767 non-null  object        
 6   drp_loc  148767 non-null  object        
 7   avg_vtt  148767 non-null  float64       
 8   avg_ctt  148767 non-null  float64       
 9   ccd_by   37191 non-null   object        
 10  rfc      37191 non-null   object        
 11  irr      8927 non-null    object        
 12  bkg_vle  148767 non-null  Int64         
 13  rid_dis  148767 non-null  float64       
 14  drv_rtg  148767 non-null  float64       
 15  cus_rtg  148767 non-null  float64       
 16  pay_mtd  101175 non-null  object        
dtypes: Int64(1), da

# Etapa 3 - Load

Esta etapa carrega o DataFrame `df_final` no banco de dados.

* Define os tipos de dados das colunas para o banco (incluindo enums e tipos numéricos/texto).
* Carrega o DataFrame usando `to_sql` com `if_exists='append'` e mapeamento de tipos.

In [20]:

logging.info("Iniciando Etapa 3: Load")

try:
    table_name = 'uber_silver'
    logging.info(f"Iniciando carregamento de {len(df_final)} linhas...")
    df_final.to_sql(
        table_name,
        engine,
        if_exists='append',
        index=False
    )
    logging.info(f"{len(df_final)} linhas carregadas com sucesso na tabela '{table_name}'!")

except Exception as e:
    logging.error(f"Erro ao carregar dados no banco: {e}")
finally:
    if 'connection' in locals() and not connection.closed:
        connection.close()
        logging.info("Conexão com o banco de dados fechada.")


2025-11-23 15:10:09,518 - INFO - Iniciando Etapa 3: Load
2025-11-23 15:10:09,520 - INFO - Iniciando carregamento de 148767 linhas...
2025-11-23 15:10:25,779 - INFO - 148767 linhas carregadas com sucesso na tabela 'uber_silver'!
